In [1]:
!pip uninstall -y edge-rec
!pip install -e .

Found existing installation: edge-rec 0.0.0
Uninstalling edge-rec-0.0.0:
  Successfully uninstalled edge-rec-0.0.0
Obtaining file:///Users/up/Desktop/CMU/Courses/10-708/project
  Preparing metadata (setup.py) ... done
  Attempting uninstall: edge-rec
    Found existing installation: edge-rec 0.0.0
    Not uninstalling edge-rec at /Users/up/Desktop/CMU/Courses/10-708/project, outside environment /Users/up/Desktop/CMU/Courses/10-708/project/venv
    Can't uninstall 'edge-rec'. No files were found to uninstall.
  Running setup.py develop for edge-rec

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
from edge_rec.gaussian_diffusion import GaussianDiffusion, Trainer
from edge_rec.model import GraphReconstructionModel

model = GraphReconstructionModel.default()

diffusion_model = GaussianDiffusion(model, image_size=10)
trainer = Trainer(diffusion_model, folder="./movie_lens", train_num_steps=int(1e4))
print("Using device:", trainer.device)

/Users/up/Desktop/CMU/Courses/10-708/project/venv/lib/python3.9/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(split_batches=True)
  warnings.warn(
Processing...
Done!


./movie_lens/processed/data.pt
Using device: mps


In [ ]:
trainer.train()

  0%|          | 0/10000 [00:00<?, ?it/s]